In [10]:
# authorize google account to use bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# authorize drive to pull model
from google.colab import drive
drive.mount('/content/drive')

# pull data for repo from bigquery
project_id = 'uic-capstone-int'
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

Authenticated
Mounted at /content/drive


In [11]:
import pandas as pd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [30]:
mongodb_repos= pd.read_csv('/content/drive/My Drive/Cloudbakers/Assets/mongodb_repos.csv')
databricks_repos= pd.read_csv('/content/drive/My Drive/Cloudbakers/Assets/databricks_repos.csv')

# mongodb_repos= pd.read_csv('mongodb_repos.csv')
# databricks_repos= pd.read_csv('databricks_repos.csv')


In [42]:
mongorepoIDs=mongodb_repos['id'].tolist()
dbrepoIDs=databricks_repos['id'].tolist()
repoIDs = mongorepoIDs + dbrepoIDs
id = repoIDs + [50904245, 2211243, 138754790, 33884891, 86057409] #list of most important repos

In [33]:
REST_commit_df = pd.read_csv('/content/drive/My Drive/Cloudbakers/Assets/REST_commit_data_for_193_repos.csv')

# Data Extraction Pipeline

In [44]:
def RESTdb(repo_id): 
    REST_df = REST_commit_df[REST_commit_df['repoID'] == repo_id]
    REST_df = REST_df[['sha','commit.committer.date','commit.committer.email']]
    REST_df['commit.committer.date'] = pd.to_datetime(REST_df['commit.committer.date'])
    REST_df['commit.committer.date'] = REST_df['commit.committer.date'].dt.date
    REST_df.rename({'commit.committer.date':'committed_date','commit.committer.email':'email', 'sha':'commit'}, axis=1, inplace=True)
    return REST_df

In [45]:
df= pd.DataFrame()

for i in range(len(id)):

    print('Reading..........repoID: %d' %id[i])
    # This cell is for data fetching process

    #-----------------------Pushes---------------------------------------------------

    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, count (*) number_of_pushes
    FROM
    (

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2015
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2016
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2017
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2018
    WHERE type='PushEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2019
    WHERE type='PushEvent' AND repo.id = {0}
    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])
    dfPushes=client.query(sql).to_dataframe()




    #---------------------------------Forks------------------------------------------

    sql='''
        
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, count (*) number_of_forks
    FROM
    (

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2015
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2016
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2017
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2018
    WHERE type='ForkEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2019
    WHERE type='ForkEvent' AND repo.id = {0}
    )
    GROUP BY 1,2,3
    ORDER BY 1;
    '''.format(id[i])
    dfForks=client.query(sql).to_dataframe()




    #-----------------------------------------Pulls------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, count (*) number_of_pulls
    FROM
    (

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2015
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2016
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2017
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2018
    WHERE type='PullRequestEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2019
    WHERE type='PullRequestEvent' AND repo.id = {0}
    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])
    dfPulls=client.query(sql).to_dataframe()



    #---------------------------------------Bookmarks-------------------------------------
    sql= '''

    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, count (DISTINCT actorID) number_of_Bookmarks
    FROM
    (
    SELECT created_at as date, repo.id as repoID, actor.id AS actorID, repo.name as repo_name
    FROM githubarchive.year.2015
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID, repo.name as repo_name
    FROM githubarchive.year.2016
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID, repo.name as repo_name
    FROM githubarchive.year.2017
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID, repo.name as repo_name
    FROM githubarchive.year.2018
    WHERE type='WatchEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, actor.id AS actorID, repo.name as repo_name
    FROM githubarchive.year.2019
    WHERE type='WatchEvent' AND repo.id = {0}
    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])
    dfBoomarks = client.query(sql).to_dataframe()






    #----------------------------Issues---------------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, count (*) number_of_issues
    FROM
    (

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2015
    WHERE type='IssuesEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2016
    WHERE type='IssuesEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2017
    WHERE type='IssuesEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2018
    WHERE type='IssuesEvent' AND repo.id = {0}

    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name
    FROM githubarchive.year.2019
    WHERE type='IssuesEvent' AND repo.id = {0}
    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])
    dfIssues=client.query(sql).to_dataframe()






    #-------------------------Issuecomments-------------------------------------------

    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, COUNT(*) AS issue_comment_count, COUNT( DISTINCT a_id) as issue_author_count
    FROM 
    (SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2015
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}


    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2016
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL



    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2017
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2018
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2019
    WHERE type = 'IssueCommentEvent' AND repo.id = {0}

    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])
    dfIssuecomment = client.query(sql).to_dataframe()



    #---------------------------------------Commitcomments------------------------------------------
    sql = '''
    SELECT FORMAT_DATETIME("%Y-%m-%d", DATETIME(date)) AS year_month, repoID, repo_name, COUNT(*) AS commit_comment_count, COUNT( DISTINCT a_id) AS commit_author_count
    FROM 
    (SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2015
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}


    UNION ALL

    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2016
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL



    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2017
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2018
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    UNION ALL


    SELECT created_at as date, repo.id as repoID, repo.name as repo_name, actor.id AS a_id
    FROM githubarchive.year.2019
    WHERE type = 'CommitCommentEvent' AND repo.id = {0}

    )
    GROUP BY 1,2,3
    ORDER BY 1;'''.format(id[i])

    dfCommitcomment = client.query(sql).to_dataframe()

    merge1 = pd.merge(dfPushes,dfForks,how='outer', on='year_month')
    merge2 = pd.merge(merge1, dfPulls, how='outer', on='year_month')
    merge3 = pd.merge(merge2, dfBoomarks, how='outer', on='year_month')
    merge4 = pd.merge(merge3, dfIssuecomment, how='outer', on='year_month')
    merge5 = pd.merge(merge4,dfCommitcomment,how='outer',on='year_month')
    merge6 = pd.merge(merge5, dfIssues, how='outer',on='year_month')
    merge6.drop(['repoID_x','repoID_y','repoID'], axis=1, inplace=True)
    merge6.drop(['repo_name_x','repo_name_y', 'repo_name'], axis=1, inplace=True)
    merge6.sort_values(by='year_month',ascending=True,inplace=True)

    merge6['year_month']=pd.to_datetime(merge6['year_month'])
    merge6.rename({'year_month': 'datetime'}, axis=1, inplace=True)
    merge6.index=merge6['datetime']
    merge6.drop('datetime',axis=1,inplace=True)

    WeeklyTS = pd.DataFrame()
    WeeklyTS = merge6.resample('W').sum()
    WeeklyTS.reset_index(level=0, inplace=True)

    repo_name_list = dfPushes['repo_name'].unique().tolist()
    repo_name_tuple = tuple(repo_name_list)

    
    dfCommits = RESTdb(id[i])
    if dfCommits.empty:

      try:
        print('Failed Commits data Extraction from REST API for repoID: %d' %id[i] + '...gathering data from BigQuery')
        sql = '''

        SELECT   commit, committer.email, DATE(TIMESTAMP_SECONDS(committer.date.seconds)) AS committed_date
        FROM `bigquery-public-data.github_repos.commits`
        WHERE EXISTS (
          SELECT 1 FROM UNNEST(repo_name) name 
          WHERE name IN {} AND EXTRACT(YEAR FROM DATE(TIMESTAMP_SECONDS(committer.date.seconds))) >2015
          )
        ORDER BY 3

        '''.format(repo_name_tuple)

        dfCommits = client.query(sql).to_dataframe() 

      except:
       print('No data from Bigquery') 
       continue
    else:
      print('Successful Commits data Extraction from REST API for repoID: %d' %id[i])


    dfCommits['committed_date']=pd.to_datetime(dfCommits['committed_date'])
    dfCommits.index=dfCommits['committed_date']
    dfCommits.drop('committed_date',axis=1,inplace=True)
    Weekly_commit_TS = pd.DataFrame()
    Weekly_commit_TS = dfCommits.resample('W').agg({'commit': 'count', 'email': 'nunique'})
    Weekly_commit_TS.rename(columns={"commit":"commit_count","email":"unique_committer_count"}, inplace=True)
    Weekly_commit_TS.reset_index(level=0, inplace=True)
    Weekly_commit_TS.rename(columns={'committed_date':'datetime'}, inplace=True)
    WeeklyTS = pd.merge(WeeklyTS,Weekly_commit_TS,how='outer', on='datetime')
    WeeklyTS['Total_count'] = WeeklyTS.drop('datetime',axis=1).sum(axis=1)


    # Setting Threshold in terms of percentage of drop
    Threshold = 0.1
    for j in range(5,len(WeeklyTS)):
      Avg = WeeklyTS['Total_count'].iloc[j-5:j,].mean()
      diff = WeeklyTS['Total_count'].iloc[j] - WeeklyTS['Total_count'].iloc[j-1]
      if diff>=0:
        WeeklyTS.loc[j,'Status'] = 1
      else:
        if abs(diff)/Avg < Threshold:
          WeeklyTS.loc[j,'Status'] = 1
        else:
          WeeklyTS.loc[j,'Status'] = 0


    WeeklyTS.index = WeeklyTS['datetime']
    WeeklyTS.drop('datetime', axis=1, inplace=True)
    WeeklyTS.insert(0, 'year', WeeklyTS.index.year)
    WeeklyTS.insert(1, 'month', WeeklyTS.index.month)
    WeeklyTS.insert(2, 'day', WeeklyTS.index.day)
    WeeklyTS.reset_index(level=0, inplace=True)
    WeeklyTS['repoID']=id[i]


    df=df.append(WeeklyTS, ignore_index=True)
df

Reading..........repoID: 156630873
Successful Commits data Extraction from REST API for repoID: 156630873
Reading..........repoID: 7567432
Successful Commits data Extraction from REST API for repoID: 7567432
Reading..........repoID: 82691991
Successful Commits data Extraction from REST API for repoID: 82691991
Reading..........repoID: 172554606
Successful Commits data Extraction from REST API for repoID: 172554606
Reading..........repoID: 183648903
Successful Commits data Extraction from REST API for repoID: 183648903
Reading..........repoID: 93444591
Successful Commits data Extraction from REST API for repoID: 93444591
Reading..........repoID: 37991233
Successful Commits data Extraction from REST API for repoID: 37991233
Reading..........repoID: 197049215
Successful Commits data Extraction from REST API for repoID: 197049215
Reading..........repoID: 194727787
Successful Commits data Extraction from REST API for repoID: 194727787
Reading..........repoID: 194736246
Successful Commits da

,datetime,year,month,day,number_of_pushes,number_of_forks,number_of_pulls,number_of_Bookmarks,issue_comment_count,issue_author_count,commit_comment_count,commit_author_count,number_of_issues,commit_count,unique_committer_count,Total_count,Status,repoID
0,2019-04-21,2019,4,21,16.0,0.0,9.0,5.0,4.0,3.0,0.0,0.0,0.0,5.0,1.0,43.0,NaN,156630873
1,2019-04-28,2019,4,28,67.0,0.0,20.0,2.0,13.0,8.0,0.0,0.0,0.0,10.0,2.0,122.0,NaN,156630873
2,2019-05-05,2019,5,5,55.0,0.0,23.0,0.0,7.0,5.0,0.0,0.0,0.0,13.0,2.0,105.0,NaN,156630873
3,2019-05-12,2019,5,12,29.0,1.0,7.0,1.0,3.0,2.0,0.0,0.0,0.0,3.0,1.0,47.0,NaN,156630873
4,2019-05-19,2019,5,19,37.0,0.0,5.0,1.0,3.0,3.0,0.0,0.0,0.0,3.0,1.0,53.0,NaN,156630873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43641,2020-09-13,2020,9,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,1.0,13.0,0.0,86057409
43642,2020-09-20,2020,9,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,86057409
43643,2020-09-27,2020,9,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,1.0,13.0,1.0,86057409
43644,2020-10-04,2020,10,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,1.0,9.0,0.0,86057409


In [52]:
df['number_of_issues'].sum(axis=0)

8979.0

In [54]:
df.to_csv("/content/drive/My Drive/Cloudbakers/Assets/Final_ds_193repos.csv")

# Commit related feature extraction using GITHUB RESTful API

* Created a new github account to generate **Access Token** which is saved in the session.py file along with the **USERNAME**. These credententials are used to make calls to the REST API

In [ ]:
import os
os.chdir('/content/drive/My Drive/Cloudbakers/Assets')

In [ ]:
import json
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import config

In [ ]:
github_api = "https://api.github.com"
gh_session = requests.Session()
gh_session.auth = (config.GITHUB_USERNAME, config.GITHUB_TOKEN)

In [ ]:
# url = github_api + '/repos/apache/spark/commits'
# commits = gh_session.get(url = url)
# commits_json = commits.json()

In [ ]:
def commits_of_repo_github(repo, owner, api):
    commits = []
    next = True
    i = 1
    while next == True:
        url = github_api + '/repos/{}/{}/commits?page={}&per_page=100'.format(owner, repo, i)
        commit_pg = gh_session.get(url = url)
        commit_pg_list = [dict(item, **{'repo_name':'{}'.format(repo)}) for item in commit_pg.json()]    
        commit_pg_list = [dict(item, **{'owner':'{}'.format(owner)}) for item in commit_pg_list]
        commits = commits + commit_pg_list
        if 'Link' in commit_pg.headers:
            if 'rel="next"' not in commit_pg.headers['Link']:
                next = False
        i = i + 1
    return commits

In [ ]:
def create_commits_df(repo, owner, api):
    commits_list = commits_of_repo_github(repo, owner, api)
    return pd.json_normalize(commits_list)

In [ ]:
commitdf=create_commits_df('jasper', 'tychoish', github_api)

In [ ]:
commitdf

,sha,node_id,url,html_url,comments_url,parents,repo_name,owner,commit.author.name,commit.author.email,commit.author.date,commit.committer.name,commit.committer.email,commit.committer.date,commit.message,commit.tree.sha,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,commit.verification.payload,author.login,author.id,author.node_id,author.avatar_url,author.gravatar_id,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.id,committer.node_id,committer.avatar_url,committer.gravatar_id,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,author,committer
0,c88adc5f621ebc6aebe1cd117ccda705ef1b715a,MDY6Q29tbWl0MTQ1NzMzNzc4OmM4OGFkYzVmNjIxZWJjNm...,https://api.github.com/repos/tychoish/jasper/c...,https://github.com/tychoish/jasper/commit/c88a...,https://api.github.com/repos/tychoish/jasper/c...,[{'sha': 'a49f332cab086a14925b057b1b130f46b18a...,jasper,tychoish,Kim Tao,kim.tao@mongodb.com,2020-04-28T17:10:32Z,GitHub,noreply@github.com,2020-04-28T17:10:32Z,MAKE-1282: fix RPC handling for time conversio...,66dfe4ab1aef543eaa03705661e39e01d315d240,https://api.github.com/repos/tychoish/jasper/g...,https://api.github.com/repos/tychoish/jasper/g...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 66dfe4ab1aef543eaa03705661e39e01d315d240\...,Kimchelly,8785663.0,MDQ6VXNlcjg3ODU2NjM=,https://avatars0.githubusercontent.com/u/87856...,,https://api.github.com/users/Kimchelly,https://github.com/Kimchelly,https://api.github.com/users/Kimchelly/followers,https://api.github.com/users/Kimchelly/followi...,https://api.github.com/users/Kimchelly/gists{/...,https://api.github.com/users/Kimchelly/starred...,https://api.github.com/users/Kimchelly/subscri...,https://api.github.com/users/Kimchelly/orgs,https://api.github.com/users/Kimchelly/repos,https://api.github.com/users/Kimchelly/events{...,https://api.github.com/users/Kimchelly/receive...,User,False,web-flow,19864447.0,MDQ6VXNlcjE5ODY0NDQ3,https://avatars3.githubusercontent.com/u/19864...,,https://api.github.com/users/web-flow,https://github.com/web-flow,https://api.github.com/users/web-flow/followers,https://api.github.com/users/web-flow/followin...,https://api.github.com/users/web-flow/gists{/g...,https://api.github.com/users/web-flow/starred{...,https://api.github.com/users/web-flow/subscrip...,https://api.github.com/users/web-flow/orgs,https://api.github.com/users/web-flow/repos,https://api.github.com/users/web-flow/events{/...,https://api.github.com/users/web-flow/received...,User,False,NaN,NaN
1,a49f332cab086a14925b057b1b130f46b18aae28,MDY6Q29tbWl0MTQ1NzMzNzc4OmE0OWYzMzJjYWIwODZhMT...,https://api.github.com/repos/tychoish/jasper/c...,https://github.com/tychoish/jasper/commit/a49f...,https://api.github.com/repos/tychoish/jasper/c...,[{'sha': '9b11f6a4e176a6e76c481a6e8cbf7554e3a9...,jasper,tychoish,julianedwards,julian.edwards@10gen.com,2020-04-24T14:08:31Z,GitHub,noreply@github.com,2020-04-24T14:08:31Z,MAKE-1197: add mrpc implementation support for...,26a3e8a07d3a6ca5b53808030b7ad424e7e4aae9,https://api.github.com/repos/tychoish/jasper/g...,https://api.github.com/repos/tychoish/jasper/g...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 26a3e8a07d3a6ca5b53808030b7ad424e7e4aae9\...,julianedwards,16562432.0,MDQ6VXNlcjE2NTYyNDMy,https://avatars2.githubusercontent.com/u/16562...,,https://api.github.com/users/julianedwards,https://github.com/julianedwards,https://api.github.com/users/julianedwards/fol...,https://api.github.com/users/ju

In [ ]:
repo_id_name = pd.read_csv('repo_id_name.csv')
repo_id_name[['owner','repo_name']] = repo_id_name.repo_name.str.split("/",expand=True)

In [ ]:
repo_id_name

,repoID,repo_name,owner
0,33628160,bson-rs,zonyitoo
1,143887688,jasper,tychoish
2,37991233,grip,tychoish
3,146502453,ftdc,tychoish
4,100294206,anser,tychoish
...,...,...,...
249,163867485,libmongocrypt,10gen
250,156630873,leafygreen-ui,10gen
251,27557391,mongo-php-library-prototype,10gen-labs
252,19514152,mongo-php-driver-prototype,10gen-labs


In [ ]:
finalcommitdf=pd.DataFrame()

for i in range(0,len(repo_id_name)):
    try:
        commitdf=create_commits_df(repo_id_name.iloc[i].repo_name, repo_id_name.iloc[i].owner, github_api)
        commitdf1=commitdf[['sha','repo_name','owner','commit.committer.date','commit.committer.email']]
        commitdf1['commit.committer.date']=pd.to_datetime(commitdf1['commit.committer.date'])
        commitdf1['repoID'] = repo_id_name.iloc[i].repoID
        finalcommitdf = finalcommitdf.append(commitdf1, ignore_index=True)
        print("{}.....Success  RepoID: {} ...... {}/{}".format(i,repo_id_name['repoID'].iloc[i] , repo_id_name['owner'].iloc[i] , repo_id_name['repo_name'].iloc[i]))
    except:
        print("{}.....Failure  RepoID: {} ...... {}/{}".format(i,repo_id_name['repoID'].iloc[i] , repo_id_name['owner'].iloc[i] , repo_id_name['repo_name'].iloc[i]))
        i+=1 #need to write query in this part to extract the data from Bigquery database for cases when repo extraction throws any exception

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.....Success  RepoID: 33628160 ...... zonyitoo/bson-rs
1.....Success  RepoID: 143887688 ...... tychoish/jasper
2.....Success  RepoID: 37991233 ...... tychoish/grip
3.....Success  RepoID: 146502453 ...... tychoish/ftdc
4.....Success  RepoID: 100294206 ...... tychoish/anser
5.....Success  RepoID: 53160128 ...... tjhunter/tensorframes
6.....Success  RepoID: 309009 ...... mongoid/mongoid
7.....Success  RepoID: 194727787 ...... mongodb/vault-plugin-secrets-mongodbatlas
8.....Success  RepoID: 232424739 ...... mongodb/vault-plugin-database-mongodbatlas
9.....Success  RepoID: 129414449 ...... mongodb/terraform_peered_vpc
10.....Success  RepoID: 194736160 ...... mongodb/terraform-provider-mongodbatlas
11.....Success  RepoID: 173968989 ...... mongodb/terraform-aws-ecs-task-definition
12.....Success  RepoID: 125097486 ...... mongodb/swift-mongoc
13.....Success  RepoID: 126176628 ...... mongodb/swift-mongo-mobile
14.....Success  RepoID: 125097454 ...... mongodb/swift-bson
15.....Success  RepoID: 

In [ ]:
finalcommitdf.head(10)

,sha,repo_name,owner,commit.committer.date,commit.committer.email,repoID
0,f6e1f5f16d5f21a9ebd756fe8d7ad4b29332e993,bson-rs,zonyitoo,2020-09-24 18:26:12+00:00,noreply@github.com,33628160
1,1636078553246cfaa373044f969279a5a331485a,bson-rs,zonyitoo,2020-09-23 17:57:29+00:00,noreply@github.com,33628160
2,432a025c3c163ef6922a11320fc75b4e0886c1ca,bson-rs,zonyitoo,2020-09-23 17:13:54+00:00,noreply@github.com,33628160
3,851f1d1a40a16e24696a0b5c04a8810af1d7e857,bson-rs,zonyitoo,2020-08-18 22:08:46+00:00,noreply@github.com,33628160
4,52528a7aa66132ebc4e35d7634429eb9af4d2192,bson-rs,zonyitoo,2020-08-07 20:46:07+00:00,noreply@github.com,33628160
5,37736669932f6894089978a9ba058a07a2263123,bson-rs,zonyitoo,2020-08-03 18:00:47+00:00,noreply@github.com,33628160
6,fd6346c48f928a186d03ff32eb8d7d3018521495,bson-rs,zonyitoo,2020-07-31 20:57:15+00:00,noreply@github.com,33628160
7,fa6e23fea9faba437ef3913e413a36571575e217,bson-rs,zonyitoo,2020-07-30 22:16:08+00:00,github@saghm.com,33628160
8,7c84ae22fb55d15391f563b57d2b9b6616a9c152,bson-rs,zonyitoo,2020-07-30 21:56:34+00:00,noreply@github.com,33628160
9,c240586a31f19d06b4a82a2280e7735832aa0c80,bson-rs,zonyitoo,2020-07-23 18:05:35+00:00,noreply@github.com,33628160


In [ ]:
finalcommitdf.to_csv("REST_commit_data_for_193_repos.csv")

In [ ]:
# # This piece of code makes calls for all the repos in 'id' list
# # dictionary might throw errors because of few empty github repos that's why exception handling in used.
# # Making unions for all the repos with weekly timestamp

# finalcommitdf=pd.DataFrame()

# for i in range(0,len(repo_id_name)):
#     try:
#         commitdf=create_commits_df(repo_id_name.iloc[i].repo_name, repo_id_name.iloc[i].owner, github_api)
#         commitdf1=commitdf[['sha','repo_name','owner','commit.committer.date','commit.committer.email']]
#         commitdf1['commit.committer.date']=pd.to_datetime(commitdf1['commit.committer.date'])
#         commitdf1.index=commitdf1['commit.committer.date']
#         commitdf2 = pd.DataFrame()
#         commitdf2['number_of_commits'] = commitdf1['sha'].resample('W').count()
#         commitdf2['number_of_unique_committers'] = commitdf1['commit.committer.email'].resample('W').nunique()
#         commitdf2['week'] = commitdf2.index
#         commitdf2['repoID'] = repo_id_name.iloc[i].repoID
#         commitdf2.reset_index(level=0, inplace=True)
#         commitdf2.drop('index', axis=1, inplace=True)
#         finalcommitdf = finalcommitdf.append(commitdf2, ignore_index=True)
#         print(i)
#     except:
#         i+=1 #need to write query in this part to extract the data from Bigquery database for cases when repo extraction throws any exception
  